# HAI Security Dataset: Model Comparison

This notebook compares the performance of different anomaly detection models trained on the HAI security dataset. We'll evaluate LSTM, Random Forest, and Autoencoder models to determine their strengths and weaknesses for industrial control system anomaly detection.

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob
import time
import joblib
from datetime import datetime
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, roc_curve, auc, f1_score, precision_score, recall_score, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import load_model
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('ggplot')
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

## 1. Load Preprocessed Test Data

First, let's load the preprocessed test data to evaluate all models on the same dataset.

In [ ]:
def load_processed_data(file_path):
    """
    Load processed data from NPZ file.
    
    Args:
        file_path: Path to the NPZ file
        
    Returns:
        DataFrame: Loaded data
    """
    # Load NPZ file
    npz_data = np.load(file_path)
    
    # Convert to DataFrame
    df = pd.DataFrame(npz_data['data'], columns=npz_data['columns'])
    
    return df

In [ ]:
# Load preprocessor
preprocessor_path = './models/hai_hai_20_07_standard_preprocessor.joblib'
preprocessor_dict = joblib.load(preprocessor_path)

# Extract important information
feature_columns = preprocessor_dict['feature_columns']
attack_columns = preprocessor_dict['attack_columns']
timestamp_col = preprocessor_dict['timestamp_col']

print(f"Number of features: {len(feature_columns)}")
print(f"Attack columns: {attack_columns}")
print(f"Timestamp column: {timestamp_col}")

In [ ]:
# Get list of test data files
test_data_dir = './processed_data/hai-20.07/test'
test_files = sorted(glob.glob(f'{test_data_dir}/*.npz'))

print(f"Test files: {[os.path.basename(f) for f in test_files]}")

In [ ]:
def load_and_prepare_tabular_data(file_paths, feature_cols, target_col=None, max_files=None, sample_fraction=None):
    """
    Load and prepare tabular data from multiple files.
    
    Args:
        file_paths: List of file paths
        feature_cols: List of feature column names
        target_col: Target column name (None for unsupervised learning)
        max_files: Maximum number of files to load (None for all files)
        sample_fraction: Fraction of data to sample (None for all data)
        
    Returns:
        tuple: (X, y) - Features and targets
    """
    all_X = []
    all_y = [] if target_col is not None else None
    
    # Limit the number of files if specified
    if max_files is not None:
        file_paths = file_paths[:max_files]
    
    for file_path in file_paths:
        print(f"Processing {os.path.basename(file_path)}...")
        
        # Load data
        df = load_processed_data(file_path)
        
        # Sample data if specified
        if sample_fraction is not None and sample_fraction < 1.0:
            df = df.sample(frac=sample_fraction, random_state=42)
        
        # Extract features
        X = df[feature_cols]
        all_X.append(X)
        
        # Extract target if provided
        if target_col is not None and target_col in df.columns:
            y = df[target_col]
            all_y.append(y)
    
    # Combine data from all files
    combined_X = pd.concat(all_X) if all_X else pd.DataFrame()
    combined_y = pd.concat(all_y) if all_y else None
    
    return combined_X, combined_y

In [ ]:
def create_sequences(data, feature_cols, target_col=None, seq_length=100):
    """
    Create sequences for LSTM input.
    
    Args:
        data: DataFrame containing the data
        feature_cols: List of feature column names
        target_col: Target column name (None for unsupervised learning)
        seq_length: Length of each sequence
        
    Returns:
        tuple: (X, y) - Sequences and targets (if target_col is provided)
    """
    X = []
    y = [] if target_col is not None else None
    
    # Extract features
    features = data[feature_cols].values
    
    # Extract target if provided
    targets = data[target_col].values if target_col is not None else None
    
    # Create sequences
    for i in range(len(features) - seq_length):
        X.append(features[i:i+seq_length])
        if target_col is not None:
            # Use the label of the last timestep in the sequence
            y.append(targets[i+seq_length])
    
    return np.array(X), np.array(y) if target_col is not None else None

In [ ]:
def load_and_prepare_sequence_data(file_paths, feature_cols, target_col=None, seq_length=100, max_files=None):
    """
    Load and prepare sequence data from multiple files.
    
    Args:
        file_paths: List of file paths
        feature_cols: List of feature column names
        target_col: Target column name (None for unsupervised learning)
        seq_length: Length of each sequence
        max_files: Maximum number of files to load (None for all files)
        
    Returns:
        tuple: (X, y) - Combined sequences and targets
    """
    all_X = []
    all_y = [] if target_col is not None else None
    
    # Limit the number of files if specified
    if max_files is not None:
        file_paths = file_paths[:max_files]
    
    for file_path in file_paths:
        print(f"Processing {os.path.basename(file_path)}...")
        
        # Load data
        df = load_processed_data(file_path)
        
        # Create sequences
        X, y = create_sequences(df, feature_cols, target_col, seq_length)
        
        all_X.append(X)
        if target_col is not None:
            all_y.append(y)
    
    # Combine data from all files
    combined_X = np.vstack(all_X) if all_X else np.array([])
    combined_y = np.concatenate(all_y) if all_y else None
    
    return combined_X, combined_y

In [ ]:
# Set parameters
target_col = 'attack' if attack_columns else None  # Target column
sample_fraction = 0.1  # Sample 10% of data to reduce memory usage
seq_length = 100  # Sequence length for LSTM

# Load tabular test data (for Random Forest and Autoencoder)
print("Loading tabular test data...")
X_test_tabular, y_test_tabular = load_and_prepare_tabular_data(test_files, feature_columns, 
                                                              target_col=target_col, 
                                                              max_files=2, 
                                                              sample_fraction=sample_fraction)

# Load sequence test data (for LSTM)
print("\nLoading sequence test data...")
X_test_sequence, y_test_sequence = load_and_prepare_sequence_data(test_files, feature_columns, 
                                                                 target_col=target_col, 
                                                                 seq_length=seq_length, 
                                                                 max_files=2)

print(f"\nTabular test data shape: {X_test_tabular.shape}, Labels shape: {y_test_tabular.shape if y_test_tabular is not None else None}")
print(f"Sequence test data shape: {X_test_sequence.shape}, Labels shape: {y_test_sequence.shape if y_test_sequence is not None else None}")

## 2. Load Trained Models

Now let's load the trained models for evaluation.

In [ ]:
# Check if models exist
model_files = {
    'lstm': './models/lstm_autoencoder_hai_20_07.h5',
    'random_forest': './models/random_forest_hai_20_07.joblib',
    'autoencoder': './models/autoencoder_hai_20_07.h5'
}

metadata_files = {
    'lstm': './models/lstm_model_metadata_hai_20_07.joblib',
    'random_forest': './models/random_forest_metadata_hai_20_07.joblib',
    'autoencoder': './models/autoencoder_metadata_hai_20_07.joblib'
}

# Check which models are available
available_models = {}
available_metadata = {}

for model_name, model_path in model_files.items():
    if os.path.exists(model_path):
        print(f"{model_name.capitalize()} model found at {model_path}")
        available_models[model_name] = model_path
    else:
        print(f"{model_name.capitalize()} model not found at {model_path}")

for model_name, metadata_path in metadata_files.items():
    if os.path.exists(metadata_path):
        print(f"{model_name.capitalize()} metadata found at {metadata_path}")
        available_metadata[model_name] = metadata_path
    else:
        print(f"{model_name.capitalize()} metadata not found at {metadata_path}")

In [ ]:
# Load models and metadata
models = {}
metadata = {}

for model_name, model_path in available_models.items():
    try:
        if model_name == 'random_forest':
            # Load scikit-learn model
            models[model_name] = joblib.load(model_path)
        else:
            # Load Keras model
            models[model_name] = load_model(model_path)
        print(f"Loaded {model_name} model successfully")
    except Exception as e:
        print(f"Error loading {model_name} model: {e}")

for model_name, metadata_path in available_metadata.items():
    try:
        metadata[model_name] = joblib.load(metadata_path)
        print(f"Loaded {model_name} metadata successfully")
    except Exception as e:
        print(f"Error loading {model_name} metadata: {e}")

## 3. Evaluate Models

Now let's evaluate each model on the test data.

In [ ]:
def evaluate_model(model_name, model, X_test, y_test, threshold=None):
    """
    Evaluate a model on test data.
    
    Args:
        model_name: Name of the model
        model: Trained model
        X_test: Test features
        y_test: Test labels
        threshold: Anomaly threshold (None to use default)
        
    Returns:
        dict: Evaluation metrics
    """
    print(f"Evaluating {model_name} model...")
    start_time = time.time()
    
    # Get predictions
    if model_name == 'lstm':
        # LSTM autoencoder reconstruction
        X_pred = model.predict(X_test)
        # Calculate reconstruction error (MSE)
        anomaly_scores = np.mean(np.square(X_test - X_pred), axis=(1, 2))
    elif model_name == 'random_forest':
        # Random Forest probability of normal class
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        # Convert to anomaly score (higher = more anomalous)
        anomaly_scores = 1 - y_pred_proba
    elif model_name == 'autoencoder':
        # Autoencoder reconstruction
        X_pred = model.predict(X_test)
        # Calculate reconstruction error (MSE)
        anomaly_scores = np.mean(np.square(X_test - X_pred), axis=1)
    else:
        raise ValueError(f"Unknown model type: {model_name}")
    
    # Use threshold from metadata if not provided
    if threshold is None and model_name in metadata:
        threshold = metadata[model_name]['threshold']
        print(f"Using threshold from metadata: {threshold}")
    elif threshold is None:
        # Use statistical threshold if no metadata
        threshold = np.mean(anomaly_scores) + 3 * np.std(anomaly_scores)
        print(f"Using statistical threshold: {threshold}")
    
    # Classify as anomaly if score > threshold
    y_pred = (anomaly_scores > threshold).astype(int)
    
    # Calculate metrics
    metrics = {
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'confusion_matrix': confusion_matrix(y_test, y_pred),
        'anomaly_scores': anomaly_scores,
        'threshold': threshold,
        'prediction_time': time.time() - start_time
    }
    
    # Calculate ROC AUC
    fpr, tpr, _ = roc_curve(y_test, anomaly_scores)
    metrics['roc_auc'] = auc(fpr, tpr)
    metrics['fpr'] = fpr
    metrics['tpr'] = tpr
    
    # Print metrics
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"ROC AUC: {metrics['roc_auc']:.4f}")
    print(f"Prediction time: {metrics['prediction_time']:.2f} seconds")
    
    return metrics

In [ ]:
# Evaluate models
evaluation_results = {}

# Evaluate LSTM model
if 'lstm' in models:
    print("\n" + "=" * 50)
    print("Evaluating LSTM Autoencoder")
    print("=" * 50)
    evaluation_results['lstm'] = evaluate_model('lstm', models['lstm'], X_test_sequence, y_test_sequence)

# Evaluate Random Forest model
if 'random_forest' in models:
    print("\n" + "=" * 50)
    print("Evaluating Random Forest")
    print("=" * 50)
    evaluation_results['random_forest'] = evaluate_model('random_forest', models['random_forest'], X_test_tabular, y_test_tabular)

# Evaluate Autoencoder model
if 'autoencoder' in models:
    print("\n" + "=" * 50)
    print("Evaluating Autoencoder")
    print("=" * 50)
    evaluation_results['autoencoder'] = evaluate_model('autoencoder', models['autoencoder'], X_test_tabular.values, y_test_tabular)

## 4. Compare Model Performance

Let's compare the performance of the different models.

In [ ]:
# Create a comparison table
if evaluation_results:
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'prediction_time']
    comparison_data = []
    
    for model_name, results in evaluation_results.items():
        row = {'Model': model_name.capitalize()}
        for metric in metrics:
            if metric in results:
                row[metric] = results[metric]
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    
    # Format the table
    pd.set_option('display.float_format', '{:.4f}'.format)
    display(comparison_df)
    
    # Reset display format
    pd.reset_option('display.float_format')
else:
    print("No evaluation results available for comparison.")

In [ ]:
# Plot comparison of key metrics
if len(evaluation_results) > 1:
    metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
    
    # Prepare data for plotting
    plot_data = []
    for model_name, results in evaluation_results.items():
        for metric in metrics_to_plot:
            if metric in results:
                plot_data.append({
                    'Model': model_name.capitalize(),
                    'Metric': metric.capitalize(),
                    'Value': results[metric]
                })
    
    plot_df = pd.DataFrame(plot_data)
    
    # Plot
    plt.figure(figsize=(14, 8))
    sns.barplot(x='Metric', y='Value', hue='Model', data=plot_df)
    plt.title('Model Performance Comparison')
    plt.ylim(0, 1.05)  # Metrics are between 0 and 1
    plt.grid(axis='y')
    plt.tight_layout()
    plt.show()
else:
    print("Need at least two models for comparison.")

In [ ]:
# Plot ROC curves for all models
plt.figure(figsize=(10, 8))

for model_name, results in evaluation_results.items():
    if 'fpr' in results and 'tpr' in results:
        plt.plot(results['fpr'], results['tpr'], 
                label=f'{model_name.capitalize()} (AUC = {results["roc_auc"]:.4f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for All Models')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
# Plot confusion matrices for all models
if evaluation_results:
    n_models = len(evaluation_results)
    fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 5))
    
    if n_models == 1:
        axes = [axes]  # Make axes iterable if only one subplot
    
    for i, (model_name, results) in enumerate(evaluation_results.items()):
        if 'confusion_matrix' in results:
            cm = results['confusion_matrix']
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
            axes[i].set_title(f'{model_name.capitalize()} Confusion Matrix')
            axes[i].set_ylabel('True Label')
            axes[i].set_xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.show()

## 5. Analyze Anomaly Score Distributions

Let's compare the anomaly score distributions for each model.

In [ ]:
# Plot anomaly score distributions
if evaluation_results:
    n_models = len(evaluation_results)
    fig, axes = plt.subplots(n_models, 1, figsize=(12, 5*n_models))
    
    if n_models == 1:
        axes = [axes]  # Make axes iterable if only one subplot
    
    for i, (model_name, results) in enumerate(evaluation_results.items()):
        if 'anomaly_scores' in results and 'threshold' in results:
            scores = results['anomaly_scores']
            threshold = results['threshold']
            
            # Get true labels
            y_true = y_test_sequence if model_name == 'lstm' else y_test_tabular
            
            # Create DataFrame for easier plotting
            score_df = pd.DataFrame({
                'Anomaly Score': scores,
                'True Label': y_true
            })
            
            # Plot distributions
            sns.histplot(data=score_df, x='Anomaly Score', hue='True Label', 
                        bins=50, kde=True, ax=axes[i])
            
            # Add threshold line
            axes[i].axvline(x=threshold, color='r', linestyle='--', 
                           label=f'Threshold = {threshold:.6f}')
            
            axes[i].set_title(f'{model_name.capitalize()} Anomaly Score Distribution')
            axes[i].legend()
    
    plt.tight_layout()
    plt.show()

## 6. Model Strengths and Weaknesses

Let's analyze the strengths and weaknesses of each model.

In [ ]:
# Create a table of model strengths and weaknesses
model_analysis = {
    'lstm': {
        'strengths': [
            'Captures temporal dependencies in the data',
            'Effective for detecting anomalies that manifest over time',
            'Can handle complex patterns in time series data'
        ],
        'weaknesses': [
            'Computationally intensive to train',
            'Requires sequence data, which increases memory usage',
            'Less interpretable than tree-based models'
        ],
        'best_for': 'Detecting anomalies that develop over time, such as gradual sensor drift or sequential attack patterns'
    },
    'random_forest': {
        'strengths': [
            'Highly interpretable with feature importance',
            'Fast prediction time',
            'Handles high-dimensional data well',
            'Robust to outliers and noise'
        ],
        'weaknesses': [
            'Does not capture temporal dependencies',
            'May overfit on small datasets',
            'Limited in capturing complex non-linear relationships'
        ],
        'best_for': 'Detecting anomalies based on feature values at a single time point, with good interpretability of which features contribute to anomalies'
    },
    'autoencoder': {
        'strengths': [
            'Learns complex non-linear relationships',
            'Effective for high-dimensional data reduction',
            'Can capture subtle patterns that simpler models might miss'
        ],
        'weaknesses': [
            'Less interpretable than tree-based models',
            'Requires careful tuning of architecture',
            'May struggle with very sparse anomalies'
        ],
        'best_for': 'Detecting complex anomalies that involve non-linear relationships between features'
    }
}

# Display analysis for available models
for model_name in evaluation_results.keys():
    if model_name in model_analysis:
        print(f"\n{model_name.upper()} ANALYSIS")
        print("=" * 50)
        print("Strengths:")
        for strength in model_analysis[model_name]['strengths']:
            print(f"- {strength}")
        print("\nWeaknesses:")
        for weakness in model_analysis[model_name]['weaknesses']:
            print(f"- {weakness}")
        print(f"\nBest for: {model_analysis[model_name]['best_for']}")

## 7. Ensemble Approach

Let's explore an ensemble approach that combines the predictions of multiple models.

In [ ]:
# Create ensemble predictions if we have multiple models
if len(evaluation_results) > 1:
    print("Creating ensemble predictions...")
    
    # Get predictions from each model
    predictions = {}
    for model_name, results in evaluation_results.items():
        if 'anomaly_scores' in results and 'threshold' in results:
            # Normalize scores to [0, 1] range for fair comparison
            scores = results['anomaly_scores']
            min_score = np.min(scores)
            max_score = np.max(scores)
            normalized_scores = (scores - min_score) / (max_score - min_score)
            
            predictions[model_name] = {
                'scores': normalized_scores,
                'binary': (scores > results['threshold']).astype(int)
            }
    
    # Create ensemble predictions using different strategies
    
    # 1. Majority voting (for binary predictions)
    if len(predictions) >= 2:
        binary_preds = np.column_stack([predictions[model]['binary'] for model in predictions])
        majority_vote = np.sum(binary_preds, axis=1) >= (len(predictions) / 2)
        
        # Evaluate majority voting
        y_true = y_test_tabular  # Use tabular labels for evaluation
        
        accuracy = accuracy_score(y_true, majority_vote)
        precision = precision_score(y_true, majority_vote)
        recall = recall_score(y_true, majority_vote)
        f1 = f1_score(y_true, majority_vote)
        
        print("\nEnsemble (Majority Voting) Results:")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        
        # Plot confusion matrix
        cm = confusion_matrix(y_true, majority_vote)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Ensemble (Majority Voting) Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
        
    # 2. Average of normalized scores
    if len(predictions) >= 2:
        score_matrix = np.column_stack([predictions[model]['scores'] for model in predictions])
        avg_scores = np.mean(score_matrix, axis=1)
        
        # Find optimal threshold
        fpr, tpr, thresholds = roc_curve(y_true, avg_scores)
        gmeans = np.sqrt(tpr * (1 - fpr))
        ix = np.argmax(gmeans)
        optimal_threshold = thresholds[ix]
        
        # Make predictions
        avg_preds = (avg_scores > optimal_threshold).astype(int)
        
        # Evaluate
        accuracy = accuracy_score(y_true, avg_preds)
        precision = precision_score(y_true, avg_preds)
        recall = recall_score(y_true, avg_preds)
        f1 = f1_score(y_true, avg_preds)
        roc_auc = auc(fpr, tpr)
        
        print("\nEnsemble (Average Scores) Results:")
        print(f"Optimal threshold: {optimal_threshold:.6f}")
        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"ROC AUC: {roc_auc:.4f}")
        
        # Plot ROC curve
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, marker='.')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.scatter(fpr[ix], tpr[ix], marker='o', color='red', 
                   label=f'Optimal (Threshold = {optimal_threshold:.6f})')
        plt.title('Ensemble (Average Scores) ROC Curve')
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend()
        plt.grid(True)
        plt.show()
        
        # Plot confusion matrix
        cm = confusion_matrix(y_true, avg_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title('Ensemble (Average Scores) Confusion Matrix')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.show()
else:
    print("Need at least two models for ensemble approach.")

## 8. Conclusion and Recommendations

Based on our analysis, we can draw the following conclusions and make recommendations for anomaly detection in industrial control systems.

### 8.1 Summary of Model Performance

Let's summarize the performance of each model and the ensemble approach:

In [ ]:
# Create a summary table with all models including ensemble if available
summary_data = []

# Add individual models
for model_name, results in evaluation_results.items():
    summary_data.append({
        'Model': model_name.capitalize(),
        'Accuracy': results.get('accuracy', np.nan),
        'Precision': results.get('precision', np.nan),
        'Recall': results.get('recall', np.nan),
        'F1 Score': results.get('f1', np.nan),
        'ROC AUC': results.get('roc_auc', np.nan)
    })

# Add ensemble if available
if 'accuracy' in locals() and 'precision' in locals() and 'recall' in locals() and 'f1' in locals() and 'roc_auc' in locals():
    summary_data.append({
        'Model': 'Ensemble (Avg)',
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROC AUC': roc_auc
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

# Format the table
pd.set_option('display.float_format', '{:.4f}'.format)
display(summary_df)

# Reset display format
pd.reset_option('display.float_format')

### 8.2 Recommendations

Based on our analysis, here are our recommendations for anomaly detection in industrial control systems:

1. **Best Overall Model**: [Will be determined based on actual results]

2. **For Real-time Detection**: Random Forest offers the fastest prediction time while maintaining good accuracy, making it suitable for real-time anomaly detection in industrial control systems.

3. **For Complex Temporal Patterns**: LSTM is recommended for detecting anomalies that manifest over time, such as gradual sensor drift or sequential attack patterns.

4. **For Interpretability**: Random Forest provides feature importance, making it easier to understand which sensors or components are contributing to anomalies.

5. **For Complex Non-linear Relationships**: Autoencoder is effective at capturing complex non-linear relationships between features.

6. **Ensemble Approach**: For critical systems where false negatives must be minimized, an ensemble approach combining multiple models can provide more robust detection.

7. **Deployment Strategy**: 
   - For edge devices with limited computational resources: Random Forest
   - For centralized monitoring systems with GPU capabilities: LSTM or Autoencoder
   - For critical infrastructure: Ensemble approach

8. **Future Work**:
   - Explore more sophisticated ensemble methods that weight models based on their performance in specific types of anomalies
   - Investigate online learning approaches to adapt to evolving normal behavior
   - Develop explainable AI techniques to better understand the nature of detected anomalies